# Human Written Text vs. AI Generated Text
### INF582
Hugo BOUIGEON, Thomas LI, Jules POTEL

-----

Ideas to try :

https://huggingface.co/docs/transformers/model_doc/gpt2:
https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

https://huggingface.co/docs/transformers/model_doc/distilbert with a classification head

## Login

In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 114.9 MB/s eta 0:00:00


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Polytechnique/INF582"

 IN582_2023_Challenge.pdf	  'submission_random forest.csv'
 INF582_final_project.ipynb	   submission_random_forest_len.csv
 logistic_regression_baseline.py   test_set.json
 submission_all_zeros.csv	   train_set.json
 submission.csv


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import csv
from sklearn.metrics import classification_report

import datasets

import transformers
print(transformers.__version__)

4.26.1


In [ ]:
# Modify
ROOT_PATH = "/content/drive/My Drive/Polytechnique/INF582/"

In [ ]:
# Read The data
training_set = pd.read_json(ROOT_PATH + 'train_set.json')
test_set = pd.read_json(ROOT_PATH + 'test_set.json')
test_set['label'] = 0 #these are fake labels, it is just easier this way to tokenize it

from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(training_set, test_size=0.2, shuffle=True, stratify=training_set['label'])

from datasets import Dataset
train_dataset = Dataset.from_pandas(train_set)
val_dataset = Dataset.from_pandas(val_set)
test_dataset = Dataset.from_pandas(test_set)

dataset = datasets.DatasetDict({"train":train_dataset,"val":val_dataset, "test":test_dataset})

In [ ]:
# Example
dataset["train"][0]

{'id': 2683,
 'text': "April 19 marks 20 years since the Oklahoma City bombing . The bombing was carried out by domestic terrorists . Today 's domestic terror threats range from eco-terrorists to anti-government extremists .",
 'label': 1,
 '__index_level_0__': 2683}

In [ ]:
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,id,text,label,__index_level_0__
0,1812,"Bradley Parkes , 16 , is fighting for his life in a coma after a suicide attempt . His mother Tiffany , 35 , shared a picture of her son in hospital . She said he had been tormented for months by a gang of teenagers in Coventry . She said : ` Every time they see him they just take everything off him ' The Manchester United fan was",0,1812
1,1286,"Fashion forecasting is a global career that focuses on upcoming trends. A fashion forecaster predicts the colors, fabrics, textures, materials, prints, graphics, beauty/grooming, accessories, footwear, street style, and other styles that will be presented on the runway and in the stores for the upcoming seasons. The concept applies to not one, but all levels of the fashion industry including haute couture, ready-to-wear, mass market, and street wear. Fashion trend forecasting is an overall process that focuses on other industries such as automobiles, medicine, food and beverages, literature, and home furnishings. Fashion forecasters are responsible for attracting consumers and helping retail businesses and designers sell their brands. Today, fashion industry workers rely on the Internet to retrieve information on new looks, colors, celebrity wardrobes, and designer collections. The fashion forecasting process includes the basic steps of understanding the vision of the business and profile of target customers, collecting information about available merchandise, preparing information, determining trends, and choosing merchandise appropriate for the company and target customer. Color and style are two of the important objects to forecast for most buyers and merchandisers.",1,1286
2,2922,Marouane Fellaini shared a glass of bubbly with his twin brother Mansour on Tuesday . The pair were recently the subject of a mix-up involving Jose Mourinho prior to Manchester United 's 3-0 defeat against Everton at Goodison Park on Sunday . The Chelsea boss prepared his players to deal with the threat of the Belgian all week . But he was told by a hotel doorman that the Belgian,0,2922
3,921,pilot reported hearing unusual banging . alaska airlines flight lands in seattle . ramp agent told authorities he fell asleep .,0,921
4,48,"Atletico Madrid 0-0 Real Madrid : Click here for Martin Samuel 's report . Atletico have a knack for replacing star players with others just as good . Jan Oblak is the latest , having replaced goalkeeper Thibaut Courtois . Mario Mandzukic looks to be a suitable replacement for Diego Costa .",1,48
5,3428,"Indianara Carvalho, 23, posted a snap of her body painted with an image of the Virgin Mary to her Instagram account. She had hymen surgery last year to 'restore' her virginity so she could enjoy her first time all over again with 'someone special'",0,3428
6,3330,"José María Ramón Obando del Campo (August 8, 1795 – April 29, 1861) was a Neogranadine General and politician who twice served as President of Colombia. As a General, he initially fought for the Royalist Army during the Independence Wars of Colombia, ultimately joining the revolutionary forces of Simón Bolívar towards the end, but once independence was attained he opposed Bolívar's Centralist government. Personal life Born out of wedlock to Ana María Crespo on August 8, 1795 in the town of Güengüe, municipality of Corinto, in the then Province of Popayán of the Viceroyalty of New Granada in present-day Colombia, he was baptised José María Ramón Iragorri Crespo just two days later on August 10 in the chapel of the García hacienda. Society, during the times of the colony, was puritanical and the religious authorities did not allow a single mother to raise a child on her own, thus when he was two years old he was given up for adoption and put in the care of a Criollo family in the home of Juan Luis Obando del Castillo y Frías and Antonia del Campo y López who raised him as their own and gave him their last name. His parentage has been of debate among historians, most argue that he wa

## Basic Analysis

### Generated Text (class = 0)

In [ ]:
n_display = 5
classe = 0
n_len = len(training_set[training_set['label'] == classe])
for idx in range(n_display):
    rand_idx = np.random.randint(n_len)
    print('index', rand_idx)
    print(training_set[training_set['label'] == classe]['text'].iloc[rand_idx])
    print('\n')

index 1053
Atif Saeed , 38 , from Lahore , Pakistan was in the city 's safari park when he spotted the male lion in the distance . Mr Saeed got out of his car and crept up towards the hunter armed only with a camera and fired off a few frames .


index 854
Honglaowai , also known as Red Laowai, are mainland Chinese men who have relationships with Taiwanese women. Honglaowai is an umbrella term that encompasses a variety of relationships, from those who have just met and are in a casual relationship to those who have been married for many years. Honglaowai is a term that encompasses a variety of relationships, from those who have just met and are in a casual relationship to those who have been married for many years. The term is often used by Taiwanese women to describe their mainland Chinese male partners, as these men often exhibit a level of understanding and respect for Taiwanese culture that is not always seen in other mainland Chinese populations. Honglaowai relationships can be f

### Written Text (class = 1)

In [ ]:
n_display = 5
classe = 1
n_len = len(training_set[training_set['label'] == classe])
for idx in range(n_display):
    rand_idx = np.random.randint(n_len)
    print('index', rand_idx)
    print(training_set[training_set['label'] == classe]['text'].iloc[rand_idx])
    print('\n')

index 1319
Al Qaeda confirms that two of its leaders were killed in January drone strikes . Other leaders have been killed or captured recently . The terrorist group 's capacity to carry out attacks in the West has been greatly diminished .


index 437
John Houlding ( – 17 March 1902) was an English businessman, most notable for being Lord Mayor of Liverpool, and the founder of Liverpool Football Club. In November 2018, Houlding was commemorated with a bronze bust outside Anfield to mark the 125th anniversary of Liverpool F.C. Biography Houlding was a businessman in the city of Liverpool. He was educated at Liverpool College and was active towards the end of the 19th century, owning a brewery that left him in a comfortable financial state for the rest of his life. He was elected to the Liverpool City Council as a Conservative representing the Everton ward, before being appointed Lord Mayor of Liverpool in 1897. He was also a member of the Orange Order, a Protestant fraternal organizati

### Simple observations

Human written text is shorter -> add length to features

A lot of repetitions on long AI generated texts


## Config

In [ ]:
# model_checkpoint = "distilbert-base-uncased"
model_checkpoint =  'roberta-base'
batch_size = 16

## Preprocessing

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
tokenizer("Hello people, I love natural lanugage processing")

{'input_ids': [0, 31414, 82, 6, 38, 657, 1632, 33532, 3252, 1580, 5774, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 13949, 753, 4863, 291, 107, 187, 5, 4020, 412, 10183, 479, 20, 10183, 21, 2584, 66, 30, 1897, 7263, 479, 2477, 128, 29, 1897, 5231, 3455, 1186, 31, 21139, 12, 26213, 1952, 7, 1475, 12, 11455, 16281, 479, 2], [0, 10169, 5272, 263, 3774, 13520, 4765, 287, 5874, 6852, 5675, 1423, 226, 1479, 26687, 13912, 34125, 36, 246, 830, 42224, 126, 365, 587, 13668, 238, 684, 18904, 25, 13520, 4765, 2424, 248, 1977, 139, 2156, 21, 10, 4938, 3390, 6, 17222, 8, 3250, 4, 590, 10, 756, 19161, 55, 87, 654, 107, 6, 79, 16, 11394, 25, 5, 78, 538, 2182, 5862, 470, 20129, 999, 11, 3049, 6, 19, 41, 3973, 756, 11, 470, 11605, 11, 5, 18283, 29, 8, 15891, 29, 4, 264, 21, 67, 1687, 65, 9, 5, 144, 505, 2182, 2415, 11, 5, 3274, 8927, 9, 4938, 11605, 4, 3011, 248, 1977, 139, 16, 67, 8715, 25, 65, 9, 5, 144, 2721, 21578, 15, 2441, 9, 70, 86, 4, 572, 145, 2967, 11, 1625, 6, 79, 880, 69, 822, 756, 11, 3049, 11, 36248, 4, 264, 56, 4502, 15, 10, 6755, 9, 1800, 3541, 6, 217, 42821, 36, 1646, 2518, 238, 35

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-classification",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [ ]:
def compute_metrics(eval_pred):
    output = {}
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    output['accuracy'] = np.mean(predictions == labels)
    return output

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/ThomasLI/roberta-base-finetuned-classification into local empty directory.


## Training

In [ ]:
# Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, id, __index_level_0__. If text, id, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3200
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
  Number of trainable parameters = 124647170
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, u

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.387013,0.828750
2,No log,0.321231,0.872500
3,0.358300,0.390982,0.860000
4,0.358300,0.666108,0.823750
5,0.149500,0.702420,0.821250


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, id, __index_level_0__. If text, id, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
Saving model checkpoint to roberta-base-finetuned-classification/checkpoint-200
Configuration saved in roberta-base-finetuned-classification/checkpoint-200/config.json
Model weights saved in roberta-base-finetuned-classification/checkpoint-200/pytorch_model.bin
tokenizer config file saved in roberta-base-finetuned-classification/checkpoint-200/tokenizer_config.json
Special tokens file saved in roberta-base-finetuned-classification/checkpoint-200/special_tokens_map.json
tokenizer config file saved in roberta-base-finetuned-classification/tokenizer_config.json
Special tokens file saved in roberta-base-finetuned-cla

TrainOutput(global_step=1000, training_loss=0.2538809814453125, metrics={'train_runtime': 1145.594, 'train_samples_per_second': 13.967, 'train_steps_per_second': 0.873, 'total_flos': 2499768803652480.0, 'train_loss': 0.2538809814453125, 'epoch': 5.0})

In [ ]:
# Evaluate the model on the validation
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, id, __index_level_0__. If text, id, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'eval_loss': 0.3212307095527649,
 'eval_accuracy': 0.8725,
 'eval_runtime': 14.8158,
 'eval_samples_per_second': 53.997,
 'eval_steps_per_second': 3.375,
 'epoch': 5.0}

In [ ]:
# Save the model
trainer.push_to_hub()

Saving model checkpoint to roberta-base-finetuned-classification
Configuration saved in roberta-base-finetuned-classification/config.json
Model weights saved in roberta-base-finetuned-classification/pytorch_model.bin
tokenizer config file saved in roberta-base-finetuned-classification/tokenizer_config.json
Special tokens file saved in roberta-base-finetuned-classification/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Mar02_14-06-44_e672abcf8e77/events.out.tfevents.1677766023.e672abcf8e77.1027.0: 100%|########…

Upload file runs/Mar02_14-06-44_e672abcf8e77/events.out.tfevents.1677767195.e672abcf8e77.1027.2: 100%|########…

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/ThomasLI/roberta-base-finetuned-classification
   76a5ea2..6c54c8d  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/ThomasLI/roberta-base-finetuned-classification
   76a5ea2..6c54c8d  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8725}]}
To https://huggingface.co/ThomasLI/roberta-base-finetuned-classification
   6c54c8d..492c0c9  main -> main

   6c54c8d..492c0c9  main -> main



'https://huggingface.co/ThomasLI/roberta-base-finetuned-classification/commit/6c54c8dc23166999f75960d054b7efd1a867a609'

In [ ]:
predictions = trainer.predict(encoded_dataset["test"])
my_pred = np.argmax(predictions.predictions, axis=1)

#Just to check 
print(my_pred.mean())

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, id. If text, id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 4000
  Batch size = 16


0.48225


## Reloading model

In [ ]:
# Load an already trained model

# trained_model_url = "ThomasLI/distilbert-base-uncased-finetuned-classification"
trained_model_url = "ThomasLI/roberta-base-finetuned-classification"
model = AutoModelForSequenceClassification.from_pretrained(trained_model_url)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ThomasLI--distilbert-base-uncased-finetuned-classification/snapshots/e8efc1348ecf93be57148c92aa290044a6effe8a/config.json
Model config DistilBertConfig {
  "_name_or_path": "ThomasLI/distilbert-base-uncased-finetuned-classification",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--ThomasLI--distilbert-base-uncased-finetuned-classification/snapshots/e8efc1348ecf93be57148c92aa290044a6effe8a/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at ThomasLI/distilbert-base-uncased-finetuned-classification.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, text, __index_level_0__. If id, text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.697126030921936,
 'eval_accuracy': 0.50375,
 'eval_runtime': 9.6613,
 'eval_samples_per_second': 82.805,
 'eval_steps_per_second': 5.175}

## Finetuning

In [ ]:
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.4 MB/s eta 0:00:00


In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"].shard(index = 1, num_shards = 10), #train on 1/10 th of the dataset
    eval_dataset=encoded_dataset["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2023-03-01 20:46:53,324] A new study created in memory with name: no-name-503da360-9df4-40bc-a85c-60ab6a4044b2
Trial: {'learning_rate': 1.2979881332007866e-05, 'num_train_epochs': 4, 'seed': 11, 'per_device_train_batch_size': 4}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.b

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.680750,0.517500
2,No log,0.650716,0.582500
3,No log,0.585816,0.682500
4,No log,0.554390,0.713750


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, text, __index_level_0__. If id, text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-classification/run-0/checkpoint-80
Configuration saved in distilbert-base-uncased-finetuned-classification/run-0/checkpoint-80/config.json
Model weights saved in distilbert-base-uncased-finetuned-classification/run-0/checkpoint-80/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-classification/run-0/checkpoint-80/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-classification/run-0/checkpoint-80/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finet

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.694017,0.500000
2,No log,0.693768,0.501250


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, text, __index_level_0__. If id, text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-classification/run-1/checkpoint-10
Configuration saved in distilbert-base-uncased-finetuned-classification/run-1/checkpoint-10/config.json
Model weights saved in distilbert-base-uncased-finetuned-classification/run-1/checkpoint-10/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-classification/run-1/checkpoint-10/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-classification/run-1/checkpoint-10/special_tokens_map.json
The following columns in the evaluation set don't have a cor

...

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, text, __index_level_0__. If id, text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-classification/run-7/checkpoint-20
Configuration saved in distilbert-base-uncased-finetuned-classification/run-7/checkpoint-20/config.json
Model weights saved in distilbert-base-uncased-finetuned-classification/run-7/checkpoint-20/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-classification/run-7/checkpoint-20/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-classification/run-7/checkpoint-20/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finet

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.684927,0.558750
2,No log,0.676575,0.602500
3,No log,0.665860,0.560000
4,No log,0.655600,0.575000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, text, __index_level_0__. If id, text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-classification/run-8/checkpoint-10
Configuration saved in distilbert-base-uncased-finetuned-classification/run-8/checkpoint-10/config.json
Model weights saved in distilbert-base-uncased-finetuned-classification/run-8/checkpoint-10/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-classification/run-8/checkpoint-10/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-classification/run-8/checkpoint-10/special_tokens_map.json
The following columns in the evaluation set don't have a cor

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.696274,0.487500
2,No log,0.695732,0.488750


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, text, __index_level_0__. If id, text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-classification/run-9/checkpoint-5
Configuration saved in distilbert-base-uncased-finetuned-classification/run-9/checkpoint-5/config.json
Model weights saved in distilbert-base-uncased-finetuned-classification/run-9/checkpoint-5/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-classification/run-9/checkpoint-5/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-classification/run-9/checkpoint-5/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-

In [ ]:
best_run

BestRun(run_id='3', objective=0.74, hyperparameters={'learning_rate': 5.894711771413252e-05, 'num_train_epochs': 5, 'seed': 30, 'per_device_train_batch_size': 4})

## Baseline Prediction

In [ ]:
# Use logistic regression to predict the class
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(training_set['text'])
# b = np.array([training_set['text'].apply(len)])
# X[:,-1] = b

c:\Users\thoma\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
clf = LogisticRegression()
clf.fit(X, training_set['label'])
X_test = vectorizer.transform(test_set['text'])
predictions = clf.predict(X_test)

In [ ]:
predictions

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
# all_zero_predictions = np.zeros(len(predictions))
# all_zero_predictions = all_zero_predictions.astype(int)
# all_zero_predictions

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(training_set['text'])
clf = RandomForestClassifier()
clf.fit(X, training_set['label'])
X_test = vectorizer.transform(test_set['text'])
predictions = clf.predict(X_test)

## Submission

In [ ]:
def submit(my_pred, name = 'last_submission'):
  # Write predictions to a file
  with open(name + ".csv", "w") as pred:
      csv_out = csv.writer(pred)
      csv_out.writerow(['id','label'])
      for i, row in enumerate(my_pred):
          csv_out.writerow([i, row])

In [ ]:
# Get the file on the left folder icon
submit(my_pred, name = 'roberta_submission')